In [1]:
!pip list | grep feat

In [101]:

from feat.utils import get_resource_path
class ResMaskNet:
    def __init__(self):
        """Initialize ResMaskNet

        @misc{luanresmaskingnet2020,
        Author = {Luan Pham & Tuan Anh Tran},
        Title = {Facial Expression Recognition using Residual Masking Network},
        url = {https://github.com/phamquiluan/ResidualMaskingNetwork},
        Year = {2020}
        }

        """
        self.transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.ToTensor()]
        )

        self.FER_2013_EMO_DICT = {
            0: "angry",
            1: "disgust",
            2: "fear",
            3: "happy",
            4: "sad",
            5: "surprise",
            6: "neutral",
        }

        # load configs and set random seed
        configs = json.load(
            open(os.path.join(get_resource_path(), "ResMaskNet_fer2013_config.json"))
        )
        self.image_size = (configs["image_size"], configs["image_size"])
        self.use_gpu = torch.cuda.is_available()
        # if self.use_gpu:
        #     self.state = torch.load(
        #         os.path.join(
        #             get_resource_path(), "ResMaskNet_Z_resmasking_dropout1_rot30.pth"
        #         )
        #     )
        # else:
        #     self.state = torch.load(
        #         os.path.join(
        #             get_resource_path(), "ResMaskNet_Z_resmasking_dropout1_rot30.pth"
        #         ),
        #         map_location={"cuda:0": "cpu"},
        #     )

        self.model = resmasking_dropout1(in_channels=3, num_classes=7)
        
        if self.use_gpu:
            self.model.load_state_dict(
                torch.load(
                    os.path.join(
                        get_resource_path(), "ResMaskNet_Z_resmasking_dropout1_rot30.pth"
                        )
                    )['net']
                )
            self.model.cuda()

        else:
            self.model.load_state_dict(
                torch.load(
                    os.path.join(
                        get_resource_path(), "ResMaskNet_Z_resmasking_dropout1_rot30.pth"
                    ),
                map_location={"cuda:0": "cpu"},
                )['net']
            )
        self.model.eval()


    def detect_emo(self, frame, detected_face, *args, **kwargs):
        """Detect emotions.

        Args:
            frame ([type]): [description]

        Returns:
            List of predicted emotions in probability: [angry, disgust, fear, happy, sad, surprise, neutral]
        """


        with torch.no_grad():
            frame = np.fliplr(frame).astype(np.uint8)
            h, w = frame.shape[:2]
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            start_x, start_y, end_x, end_y, conf = np.array(detected_face[0]).astype(
                int
            )
            # covnert to square images
            center_x, center_y = (start_x + end_x) // 2, (start_y + end_y) // 2
            square_length = ((end_x - start_x) + (end_y - start_y)) // 2 // 2
            square_length *= 1.1
            start_x = int(center_x - square_length)
            start_y = int(center_y - square_length)
            end_x = int(center_x + square_length)
            end_y = int(center_y + square_length)
            if start_x < 0:
                start_x = 0
            if start_y < 0:
                start_y = 0
            face = gray[start_y:end_y, start_x:end_x]
            face = ensure_color(face)
            face = cv2.resize(face, self.image_size)
            if self.use_gpu:
                face = self.transform(face).cuda()
            else:
                face = self.transform(face)
            face = torch.unsqueeze(face, dim=0)
            output = torch.squeeze(self.model(face), 0)
            proba = torch.softmax(output, 0)
            proba_np = proba.cpu().numpy()
            return [proba_np]

ModuleNotFoundError: No module named 'feat'

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
# from torch.nn.parallel import DistributedDataParallel
from resmasknet_test import *
import random
from tqdm import tqdm
from PIL import Image
import cv2
import numpy as np
from tensorboardX import SummaryWriter
import json
# from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from human_data import feat_order

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(1)



/opt/conda/envs/pybert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class SiameseRankNet(nn.Module):
    def __init__(self, model_path):
        super(SiameseRankNet, self).__init__()
        # Load ResMaskNet model
        # self.model = resmasking_dropout1(in_channels=3, num_classes=7)
        self.model = ResMasking('')
        # freeze
        # for param in self.model.parameters():
        #     param.requires_grad = False
        # unfreeze
        # Define the fully connected layers on top of concatenated feature vectors
        self.model.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 7)
        )
        
        self.use_gpu = torch.cuda.is_available()
        self.image_size = 224

        self.FER_2013_EMO_DICT = {
            0: "angry",
            1: "disgust",
            2: "fear",
            3: "happy",
            4: "sad",
            5: "surprise",
            6: "neutral",
        }
        self.FER_2013_EMONUM = {v:k for k, v in self.FER_2013_EMO_DICT.items()}
        self.emotion = 'happy'
        self.idx = self.FER_2013_EMONUM[self.emotion]
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        self.sigmoid = nn.Sigmoid()
        # self.activation = nn.Tanh()
        # self.dropout = nn.Dropout(p=0.5)
        # self.relu = nn.ReLU()
        self.model_path = model_path
        
        
        
        
        if self.use_gpu:
            self.state_dict = torch.load(self.model_path)
            for key in list(self.state_dict.keys()):
                self.state_dict[key.replace("module.model.","")] = self.state_dict.pop(key)
            self.model.load_state_dict(self.state_dict)
            self.model.cuda()

        else:
            self.state_dict = torch.load(self.model_path, map_location={"cuda:0": "cpu"},)
            for key in list(self.state_dict.keys()):
                self.state_dict[key.replace("module.model.","")] = self.state_dict.pop(key)
            self.model.load_state_dict()
        self.model.eval()
        
    def detect_emo(self, frame, detected_face="", *args, **kwargs):
        """Detect emotions.

        Args:
            frame ([type]): [description]

        Returns:
            List of predicted emotions in probability: [angry, disgust, fear, happy, sad, surprise, neutral]
        """

        with torch.no_grad():
            # frame = np.fliplr(frame).astype(np.uint8)
            # h, w = frame.shape[:2]
            # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # start_x, start_y, end_x, end_y, conf = np.array(detected_face[0]).astype(
            #     int
            # )

            # test
            start_x, start_y, end_x, end_y = 193, 114, 442, 363

            # # covnert to square images
            # center_x, center_y = (start_x + end_x) // 2, (start_y + end_y) // 2
            # square_length = ((end_x - start_x) + (end_y - start_y)) // 2 // 2
            # square_length *= 1.1
            # start_x = int(center_x - square_length)
            # start_y = int(center_y - square_length)
            # end_x = int(center_x + square_length)
            # end_y = int(center_y + square_length)
            # if start_x < 0:
            #     start_x = 0
            # if start_y < 0:
            #     start_y = 0
            # face = gray[start_y:end_y, start_x:end_x]
            # face = ensure_color(face)
            # face = cv2.resize(face, (self.image_size, self.image_size))
    
            face = frame.crop([start_x, start_y, end_x, end_y])


            
            
            if self.use_gpu:
                face = self.transform(face).cuda()
            else:
                face = self.transform(face)
            face = torch.unsqueeze(face, dim=0)
            output = torch.squeeze(self.model(face), 0)
            # proba = torch.softmax(output, 0)
            # proba_np = proba.cpu().numpy()
            return output



path1 = "check_points/Happy_Rank1_50_v1/20230809_170453/model_20230809_170453_epoch2.pt"
model = SiameseRankNet(path1)
# model2 = ResMaskNet()

# print(model.detect_emo(Image.open('data/happiness_selected_imgonly100/ha_1.png')))


In [10]:
model.model.fc

Sequential(
  (0): Dropout(p=0.3, inplace=False)
  (1): Linear(in_features=512, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=7, bias=True)
)

In [15]:
aver_4_sorted_data = ['ha_428.png', 'ha_393.png', 'ha_407.png', 'ha_422.png', 'ha_341.png', 'ha_202.png', 'ha_235.png', 'ha_288.png', 'ha_50.png', 'ha_489.png', 'ha_212.png', 'ha_450.png', 'ha_348.png', 'ha_24.png', 'ha_443.png', 'ha_256.png', 'ha_412.png', 'ha_185.png', 'ha_301.png', 'ha_11.png', 'ha_201.png', 'ha_497.png', 'ha_384.png', 'ha_27.png', 'ha_184.png', 'ha_368.png', 'ha_268.png', 'ha_442.png', 'ha_491.png', 'ha_367.png', 'ha_311.png', 'ha_19.png', 'ha_243.png', 'ha_249.png', 'ha_215.png', 'ha_107.png', 'ha_130.png', 'ha_221.png', 'ha_303.png', 'ha_342.png', 'ha_244.png', 'ha_2.png', 'ha_54.png', 'ha_320.png', 'ha_32.png', 'ha_329.png', 'ha_241.png', 'ha_315.png', 'ha_33.png', 'ha_451.png', 'ha_151.png', 'ha_5.png', 'ha_476.png', 'ha_296.png', 'ha_344.png', 'ha_1.png', 'ha_152.png', 'ha_132.png', 'ha_172.png', 'ha_207.png', 'ha_377.png', 'ha_131.png', 'ha_404.png', 'ha_149.png', 'ha_477.png', 'ha_466.png', 'ha_454.png', 'ha_186.png', 'ha_122.png', 'ha_106.png', 'ha_124.png', 'ha_43.png', 'ha_292.png', 'ha_380.png', 'ha_284.png', 'ha_59.png', 'ha_110.png', 'ha_396.png', 'ha_382.png', 'ha_475.png', 'ha_334.png', 'ha_119.png', 'ha_482.png', 'ha_381.png', 'ha_9.png', 'ha_252.png', 'ha_174.png', 'ha_145.png', 'ha_116.png', 'ha_314.png', 'ha_128.png', 'ha_310.png', 'ha_262.png', 'ha_257.png', 'ha_139.png', 'ha_84.png', 'ha_233.png', 'ha_399.png', 'ha_111.png', 'ha_16.png']

# test for pyfeat ranking
cleaned_data = feat_order['happy'][:5]
cleaned_data = feat_order['happy']

final_res = []
for i in cleaned_data:
    tmp_path = os.path.join('data/happiness_selected_imgonly100', i)
    tmp_res = {}
    for k,v in model.FER_2013_EMO_DICT.items():
        # print(k)
        # print(model.detect_emo(Image.open(tmp_path)))
        tmp_res[v] = list(model.detect_emo(Image.open(tmp_path)))[k]
    print(tmp_res['happy'])
    final_res.append(tmp_res['happy'].tolist())
    # print(tmp_res)
    # print()
    


tensor(-0.0516, device='cuda:0')
tensor(-0.0038, device='cuda:0')
tensor(-0.1297, device='cuda:0')
tensor(-0.0667, device='cuda:0')
tensor(0.0272, device='cuda:0')
tensor(-0.0012, device='cuda:0')
tensor(-0.0682, device='cuda:0')
tensor(-0.0259, device='cuda:0')
tensor(-0.0088, device='cuda:0')
tensor(-0.0513, device='cuda:0')
tensor(0.0209, device='cuda:0')
tensor(-0.0256, device='cuda:0')
tensor(-0.1062, device='cuda:0')
tensor(-0.0513, device='cuda:0')
tensor(-0.0970, device='cuda:0')
tensor(-0.0205, device='cuda:0')
tensor(-0.1076, device='cuda:0')
tensor(-0.0797, device='cuda:0')
tensor(-0.1449, device='cuda:0')
tensor(-0.0703, device='cuda:0')
tensor(0.0256, device='cuda:0')
tensor(-0.0369, device='cuda:0')
tensor(0.1354, device='cuda:0')
tensor(-0.0338, device='cuda:0')
tensor(-0.1594, device='cuda:0')
tensor(-0.0220, device='cuda:0')
tensor(-0.1737, device='cuda:0')
tensor(0.0482, device='cuda:0')
tensor(-0.1014, device='cuda:0')
tensor(-0.1305, device='cuda:0')
tensor(-0.1469,

In [22]:

final_rank = [[i, v] for i, v in enumerate(final_res)]

final_rank.sort(key=lambda x:x[1], reverse=True)

print(final_rank)
fin = [aver_4_sorted_data[i[0]] for i in final_rank]
print(fin)

[[22, 0.13539372384548187], [27, 0.048190079629421234], [45, 0.03191537782549858], [4, 0.027175672352313995], [20, 0.025602273643016815], [10, 0.02091219276189804], [34, 0.002751395106315613], [5, -0.0011971443891525269], [38, -0.0032579824328422546], [1, -0.0037516728043556213], [43, -0.005445666611194611], [8, -0.008805736899375916], [31, -0.012804258614778519], [15, -0.020520687103271484], [25, -0.0219891294836998], [11, -0.02563406527042389], [7, -0.025892414152622223], [23, -0.033829618245363235], [21, -0.036931782960891724], [50, -0.04082949087023735], [9, -0.051255807280540466], [13, -0.05126189440488815], [0, -0.05163617432117462], [3, -0.06668658554553986], [6, -0.06819435209035873], [19, -0.07032498717308044], [17, -0.07974196970462799], [14, -0.0970403403043747], [73, -0.10070277750492096], [39, -0.10132899880409241], [28, -0.10137064009904861], [12, -0.10623828321695328], [16, -0.10763641446828842], [44, -0.12455587834119797], [2, -0.12971627712249756], [29, -0.130534887313

In [94]:
# test for pyfeat ranking
cleaned_data = feat_order['happy'][-5:]

for i in cleaned_data:
    tmp_path = os.path.join('data/happiness_selected_imgonly100', i)
    tmp_res = {}
    for k,v in model.FER_2013_EMO_DICT.items():
        # print(k)
        # print(model.detect_emo(Image.open(tmp_path)))
        tmp_res[v] = list(model.detect_emo(Image.open(tmp_path)))[k]
    print(tmp_res['happy'])
    print()
    

0.09543101

0.09662038

0.10928758

0.10535402

0.090404704



In [95]:
tmp_res

{'angry': 0.07773054,
 'disgust': 0.21724878,
 'fear': 0.22108388,
 'happy': 0.090404704,
 'sad': 0.082979575,
 'surprise': 0.17381632,
 'neutral': 0.13673624}

In [96]:
sum(tmp_res.values())

1.0000000447034836